In [ ]:
#On peut passer à l'étape suivante

#PARTIE 2/ ANALYSE STATISTIQUE

In [ ]:
#Pour faire le lien entre la Partie 1 et 2
data = df

In [ ]:
# Ici, nous calculons les statistiques descriptives pour toutes les valeurs numériques

numerical_cols = data.select_dtypes(include=[np.number]).columns
for col in numerical_cols:
  if col != 'PassengerId': #On exclut le numéro de passager
    print(f"\n**Descriptive statistics of {col}**")
    print(data[col].describe())  # Affichons les résultats

In [ ]:
#Représentons graphiquement la distribution des variables.

features = ['Sex', 'Age', 'Fare', 'Parch', 'SibSp', 'Survived', 'Pclass', 'Embarked']

# On crée un histogramme pour chaque variable

fig, axes = plt.subplots(2, 4, figsize=(12, 6))

for i, feature in enumerate(features):
    row, col = i // 4, i % 4
    sns.histplot(data[feature], ax=axes[row, col])
    axes[row, col].set_title(feature)

# Pour que les graphiques ne s'emboitent pas
plt.tight_layout()

# Affichons les grpahiques
plt.show()

In [ ]:
#On observe qu'il y a le double de passagers masculins que de passagers féminins.

#La distribution des passagers selon l'age nous semble approximativement normale.

#La distribution des prix des billets nous semble lognormale. En effet il y a une forte skewness à gauche.

#Les variables Parch et SibSp ont toutes les deux un mode à 0 et, environ 1/4 des observations au dessus de 0.

#Environ 60% pourcent des passagers n'ont pas survécu au naufrage.

#La distribution des classes est proche de: 50% des passagers en classe 3 et 25% pour chacune des classes 1 et 2.

#Environ 2/3 des passagers ont embarqué à Southampton, 20% à Cherbourg et le reste à Queensland.

In [ ]:

# Pour la visualisation de la survie des passagers, selon différentes variables.
categorical_features = ['Parch', 'SibSp', 'Embarked', 'Sex','Pclass']

# On crée un graphique pour la survie, selon chacune des variables ci-dessus
for feature in categorical_features:
    sns.barplot(x=feature, y='Survived', data=data)
    plt.title(f"Probabilités de survie selon {feature}")
    plt.xlabel(feature)
    plt.ylabel("Taux de survie")
    plt.show()

# Pour une représentation numérique des graphiques 
for feature in categorical_features:
    print(f"Probabilités de survie selon {feature}:")
    print(data.groupby(feature)['Survived'].mean())
    print("\n")

In [ ]:
#Le taux de survie selon parch est moyen pour une valeur de 0, inférieur à la moyenne pour pour une Valeur de 5. Les autres valeurs on une probabilité de survie  de 10 à 20% au dessus de la moyenne.
#Il semble donc que le fait d'appartenir à une famille de taille moyenne augmente les probabilités de survie, vis-à-vis de ménages sans enfants et de grandes familles.

#La variable SibSp semble corroborer ceci.

#En ce qui concerne le port d'embarquement, il sembe que les passagers ayant embarqué à Cherbourg ont une plus forte probabilité de survie que les autres passagers.

#Le fait d'être de sexe féminin confère un significatif avantage en termes de survie. En effet, près de 80% des passagers féminins ont survécu au naufrage, vs. moins de 20% des passagers masculins.

#La probabilité de survie des passagers de classe 3 est en dessous de la moyennene. Les passagers de classe 2 ont près de deux fois plus de chance de survivre, et les passagers de classe 1 ont plus de 60% de chance de survivre.

In [ ]:
#Les variables 'Fare' et 'Age' sont difficilement représentables en tant que telles.
#Il va nous falloir les condenser en classes

In [ ]:
#Ici, nous représentons la survie des passagers, selon différentes classes de prix de ticket

# On segmente les prix des tickets (une variable continue) en seulement 5 classes.
def fare_band(fare):
    fare_bands = [0, 50, 100, 200, 1000]
    for i in range(len(fare_bands) - 1):
        if fare_bands[i] <= fare < fare_bands[i + 1]:
            return f"{fare_bands[i]} ~ {fare_bands[i + 1]}"
    return f"{fare_bands[-1]}+"

# Créons une nouvelle colonne pour les classes
data['fare_band'] = data['Fare'].apply(fare_band)

# Calculons la chance de survie, par classe
titanic_fare = data[['fare_band', 'Survived']].groupby('fare_band')['Survived'].value_counts().sort_index().unstack().fillna(0)
titanic_fare['Survival rate'] = titanic_fare[1] / (titanic_fare[0] + titanic_fare[1]) * 100


# Définissons les couleurs
color_map = ['#d4dddd' for _ in range(5)]
color_map[0] = color_map[4] = '#244747'

# Créons le graphique
fig, ax = plt.subplots(1, 1, figsize=(10, 7))

ax.bar(
    titanic_fare['Survival rate'].index,
    titanic_fare['Survival rate'],
    color=color_map,
    width=0.55,
    edgecolor='black',
    linewidth=0.7
)

for s in ["top", "right", "left"]:
    ax.spines[s].set_visible(False)

mean = data['Survived'].mean() * 100
ax.axhline(mean, color='black', linewidth=0.4, linestyle='dashdot')
ax.annotate(
    f"Moyenne : {mean :.4}%",
    xy=('200 ~ 1000', mean + 4),
    va='center',
    ha='center',
    color='#4a4a4a',
    bbox=dict(boxstyle='round', pad=0.4, facecolor='#efe8d1', linewidth=0)
)

grid_y_ticks = np.arange(0, 101, 20)
ax.set_yticks(grid_y_ticks)
ax.grid(axis='y', linestyle='-', alpha=0.4)

# Annotons le graphique
for i in titanic_fare['Survival rate'].index:
    ax.annotate(
        f"{titanic_fare['Survival rate'][i]:.02f}%",
        xy=(i, titanic_fare['Survival rate'][i] + 2.3),
        va='center',
        ha='center',
        fontweight='light',
        color='#4a4a4a'
    )

fig.text(0.06, 1, 'Probabilités de survie selon le prix du billet', fontsize=15, fontweight='bold', fontfamily='serif')
fig.text(0.06, 0.96, "Il semble qu'il y ait un lien important entre les chances de survie et le prix du billet", fontsize=12, fontweight='light', fontfamily='serif')


plt.tight_layout()
plt.show()


In [ ]:

def age_band(num):
    for i in range(1, 100):
        if num < 10*i :  return f'{(i-1) * 10} ~ {i*10}'

data['age_band'] = data['Age'].apply(age_band)
titanic_age = data[['age_band', 'Survived']].groupby('age_band')['Survived'].value_counts().sort_index().unstack().fillna(0)
titanic_age['Survival rate'] = titanic_age[1] / (titanic_age[0] + titanic_age[1]) * 100

fig, ax = plt.subplots(1, 1, figsize=(10, 7))

color_map = ['#d4dddd' for _ in range(9)]
color_map[0] = color_map[8] = '#244747' # color highlight

ax.bar(titanic_age['Survival rate'].index, titanic_age['Survival rate'], 
       color=color_map, width=0.55, 
       edgecolor='black', 
       linewidth=0.7)



for s in ["top","right","left"]:
    ax.spines[s].set_visible(False)

grid_y_ticks = np.arange(0, 101, 20)
ax.set_yticks(grid_y_ticks)
ax.grid(axis='y', linestyle='-', alpha=0.4)


# Annotons le graphique
for i in titanic_age['Survival rate'].index:
    ax.annotate(f"{titanic_age['Survival rate'][i]:.02f}%", 
                   xy=(i, titanic_age['Survival rate'][i] + 2.3),
                   va = 'center', ha='center',fontweight='light', 
                   color='#4a4a4a')


mean = data['Survived'].mean() *100
ax.axhline(mean ,color='black', linewidth=0.4, linestyle='dashdot')
ax.annotate(f"Moyenne : {mean :.4}%", 
            xy=('70 ~ 80', mean + 4),
            va = 'center', ha='center',
            color='#4a4a4a',
            bbox=dict(boxstyle='round', pad=0.4, facecolor='#efe8d1', linewidth=0))
    

fig.text(0.06, 1, "Probabilités de survie selon lâge", fontsize=15, fontweight='bold', fontfamily='serif')
fig.text(0.06, 0.96, 'On voit que le taux de survie est nettement plus élevé chez les plus jeunes et les plus agés.', fontsize=12, fontweight='light', fontfamily='serif')


plt.tight_layout()
plt.show()

In [ ]:

# Sur ce scatterplot on représente la distribution des prix de tickets selon l'age des passagers

plt.figure(figsize=(8, 6))
plt.scatter(data['Age'], data['Fare'], alpha=0.5)
plt.xlabel('Age')
plt.ylabel('Fare')
plt.title('Age vs. Fare Distribution')
plt.grid(True)
plt.show()


In [ ]:
#Ensuite, on va chercher à examiner la relation entre la survie des passagers, et plusieurs variables en même temps.

In [ ]:
#Ici, nous voyons la survie selon le sexe ainsi que la classe de voyage des passagers

#Premièrement, on observe que, quelle que soit la classe, les passagers de sexe féminin ont des chances de survie nettement meilleures.

#Ensuite, on constate que les chances de survie des femmes sont à peu près équivalentes pour les classes 1 et 2, mais sont quasiment divisées par 2  pour la classe 3.

#De plus, les chances de survie des hommes voyageant en classe 1 et la survie des femmes voyageant en classe 3 sont à peu près équivalents. Néanmoins, ces chances de survie sont divsées par 2 pour les classes 2 et 3.

ax = sns.catplot(
        x='Pclass',
        y='Survived',
        hue='Sex',
        kind='bar',
        data=data,
    )

In [ ]:
#Ici, nous voyons les probabilités de survie selon la classe de passager et le port d'embarquement.

#Comme prévu, ces chances sont décroissantes avec la classe (1 étant la classe prémium et 3 la classe la plus modeste)

#On remarque également qu'en moyenne, les passagers embarqués à Southhampton ont de nettement plus faibles probabilités de survie que les passagers embarqués dans les autres ports.

with sns.plotting_context('notebook',font_scale=1):

    ax = sns.catplot(
        x='Pclass',
        y='Survived',
        hue='Embarked',
        kind='bar',
        data=data,
    )

In [ ]:
with sns.plotting_context('notebook',font_scale=1.2):

    ax = sns.catplot(
        x='Survived',
        y='Age',
        hue='Sex',
        col='Pclass',
        data=data,
        orient='v',
        kind='boxen',
    )